In [9]:
import torch
import torch.nn as nn

from torchvision.transforms import ToTensor, RandomAffine, RandomHorizontalFlip, RandomCrop

from data import combine_transforms, load_data, OneHot

import numpy as np

In [10]:
class ConvBlock(nn.Module):
    
    def __init__(self, n_filters):
        
        super(ConvBlock, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(n_filters, n_filters, 3, padding=1),
            nn.BatchNorm2d(n_filters),
            nn.ReLU(),
            nn.Conv2d(n_filters, n_filters, 3, padding=1),
            nn.BatchNorm2d(n_filters)
        )
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        out = self.conv(x)
        
        out = x + out
        
        out = self.relu(out)
        
        return out
    
class DownsampleConvBlock(nn.Module):
    
    def __init__(self, n_filters):
        
        super(DownsampleConvBlock, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(n_filters, 2*n_filters, 3, padding=1, stride=2),
            nn.BatchNorm2d(2*n_filters),
            nn.ReLU(),
            nn.Conv2d(2*n_filters, 2*n_filters, 3, padding=1),
            nn.BatchNorm2d(2*n_filters)
        )
        
        self.proj = nn.Conv2d(n_filters, 2*n_filters, 1, stride=2)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        
        out = self.conv(x)
        
        x = self.proj(x)
        
        out = x + out
        
        out = self.relu(out)
        
        return out
        

class Classififer(nn.Module):
    
    def block(self, in_filters, n_layers=3, down=True):
        
        layers = []
        
        if down:
            layers.append(DownsampleConvBlock(in_filters))
            n_filters = 2 * in_filters
        else:
            layers.append(ConvBlock(in_filters))
            n_filters = in_filters
            
        for _ in range(n_layers - 1):
            layers.append(ConvBlock(n_filters))
            
        return nn.Sequential(*layers)
    
    def __init__(self):
        
        super(Classififer, self).__init__()
        
        n = 2
        
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            self.block(16, n_layers=2*n, down=False),
            self.block(16, n_layers=2*n),
            self.block(32, n_layers=2*n)
        )
        
        self.pool = nn.AdaptiveAvgPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Linear(64, 10), nn.Softmax(1)
        )
        
    def forward(self, x):
        x = self.conv(x)

        x = self.pool(x)
        
        x = x.reshape(-1, 64)
        
        x = self.fc(x)
        
        return x

In [11]:
use_gpu = True

n_epochs = 50
batch_size = 100

criterion = torch.nn.CrossEntropyLoss()

In [17]:
prep = []
# aug = [RandomAffine(30), RandomHorizontalFlip()]
# aug = [RandomCrop(32, padding=4), RandomHorizontalFlip()]
aug = [RandomAffine(30), RandomCrop(32, padding=4), RandomHorizontalFlip()]
postp = [ToTensor()]
target = []

train_transform, test_transform, target_transform = combine_transforms(preprocessing=prep, 
                                                                        augmentations=aug, 
                                                                        postprocessing=postp, 
                                                                        target=target)

In [18]:
train_loader, test_loader = load_data('../data', 
                                      train_transform, test_transform, target_transform, 
                                      batch_size, use_gpu)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
def initialize_model(use_gpu=True):
    model = Classififer()
    
    if use_gpu:
        model = model.cuda()
        
    return model

def train_epoch(model, criterion, loader, optimizer):
    
    model.train()
    
    losses = []
    accs = []
    
    for batch in loader:

        x, y = batch
        
        y_pred = model(x)
        
        loss = criterion(y_pred, y)
        accuracy = torch.mean((torch.argmax(y_pred, dim=1) == y).float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        accs.append(accuracy.item())
    
    return np.mean(losses), np.mean(accs)

def evaluate_model(model, criterion, loader):
    
    model.eval()
    
    losses = []
    accs = []

    for batch in loader:

        x, y = batch

        y_pred = model(x)

        loss = criterion(y_pred, y)
        accuracy = torch.mean((torch.argmax(y_pred, dim=1) == y).float())
        
        losses.append(loss.item())
        accs.append(accuracy.item())
    
    return np.mean(losses), np.mean(accs)

def train_model(model, criterion, train_loader, val_loader, n_epochs, optimizer):
    
    for epoch in range(n_epochs):
        
        loss, acc = train_epoch(model, criterion, train_loader, optimizer)
        
        val_loss, val_acc = evaluate_model(model, criterion, val_loader)
        
        print(f'epoch #{epoch} loss: {loss:.3f}, acc: {acc:.3f}, val_loss: {val_loss:.3f}, val_acc: {val_acc:.3f}')

In [20]:
model = initialize_model(
    use_gpu=use_gpu
)

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

In [8]:
train_model(
    model, 
    criterion,
    train_loader, 
    test_loader,
    n_epochs,
    optimizer
)

epoch #0 loss: 2.120, acc: 0.342, val_loss: 2.025, val_acc: 0.442
epoch #1 loss: 2.015, acc: 0.448, val_loss: 1.991, val_acc: 0.471
epoch #2 loss: 1.977, acc: 0.486, val_loss: 1.983, val_acc: 0.475
epoch #3 loss: 1.951, acc: 0.513, val_loss: 1.930, val_acc: 0.533
epoch #4 loss: 1.929, acc: 0.536, val_loss: 1.932, val_acc: 0.530
epoch #5 loss: 1.908, acc: 0.556, val_loss: 1.892, val_acc: 0.569
epoch #6 loss: 1.890, acc: 0.575, val_loss: 1.863, val_acc: 0.596
epoch #7 loss: 1.876, acc: 0.588, val_loss: 1.868, val_acc: 0.594
epoch #8 loss: 1.865, acc: 0.598, val_loss: 1.846, val_acc: 0.615
epoch #9 loss: 1.855, acc: 0.607, val_loss: 1.835, val_acc: 0.627
epoch #10 loss: 1.844, acc: 0.619, val_loss: 1.826, val_acc: 0.636
epoch #11 loss: 1.834, acc: 0.627, val_loss: 1.815, val_acc: 0.646
epoch #12 loss: 1.828, acc: 0.635, val_loss: 1.797, val_acc: 0.665
epoch #13 loss: 1.819, acc: 0.644, val_loss: 1.801, val_acc: 0.663
epoch #14 loss: 1.812, acc: 0.652, val_loss: 1.792, val_acc: 0.672
epoch

In [16]:
train_model(
    model, 
    criterion,
    train_loader, 
    test_loader,
    n_epochs,
    optimizer
)

epoch #0 loss: 2.142, acc: 0.314, val_loss: 2.065, val_acc: 0.398
epoch #1 loss: 2.011, acc: 0.455, val_loss: 1.987, val_acc: 0.475
epoch #2 loss: 1.950, acc: 0.517, val_loss: 1.938, val_acc: 0.523
epoch #3 loss: 1.914, acc: 0.553, val_loss: 1.909, val_acc: 0.551
epoch #4 loss: 1.884, acc: 0.582, val_loss: 1.896, val_acc: 0.567
epoch #5 loss: 1.865, acc: 0.601, val_loss: 1.889, val_acc: 0.571
epoch #6 loss: 1.847, acc: 0.618, val_loss: 1.850, val_acc: 0.610
epoch #7 loss: 1.832, acc: 0.633, val_loss: 1.859, val_acc: 0.604
epoch #8 loss: 1.821, acc: 0.643, val_loss: 1.829, val_acc: 0.633
epoch #9 loss: 1.809, acc: 0.654, val_loss: 1.806, val_acc: 0.654
epoch #10 loss: 1.798, acc: 0.666, val_loss: 1.789, val_acc: 0.674
epoch #11 loss: 1.786, acc: 0.678, val_loss: 1.795, val_acc: 0.669
epoch #12 loss: 1.778, acc: 0.686, val_loss: 1.783, val_acc: 0.680
epoch #13 loss: 1.770, acc: 0.695, val_loss: 1.804, val_acc: 0.656
epoch #14 loss: 1.762, acc: 0.702, val_loss: 1.768, val_acc: 0.692
epoch

In [21]:
train_model(
    model, 
    criterion,
    train_loader, 
    test_loader,
    n_epochs,
    optimizer
)

epoch #0 loss: 2.149, acc: 0.306, val_loss: 2.078, val_acc: 0.377
epoch #1 loss: 2.052, acc: 0.408, val_loss: 2.016, val_acc: 0.440
epoch #2 loss: 2.008, acc: 0.454, val_loss: 1.985, val_acc: 0.476
epoch #3 loss: 1.979, acc: 0.484, val_loss: 1.958, val_acc: 0.501
epoch #4 loss: 1.960, acc: 0.503, val_loss: 1.935, val_acc: 0.523
epoch #5 loss: 1.937, acc: 0.527, val_loss: 1.913, val_acc: 0.549
epoch #6 loss: 1.924, acc: 0.538, val_loss: 1.897, val_acc: 0.565
epoch #7 loss: 1.908, acc: 0.556, val_loss: 1.900, val_acc: 0.559
epoch #8 loss: 1.897, acc: 0.566, val_loss: 1.871, val_acc: 0.588
epoch #9 loss: 1.888, acc: 0.575, val_loss: 1.881, val_acc: 0.579
epoch #10 loss: 1.878, acc: 0.584, val_loss: 1.864, val_acc: 0.597
epoch #11 loss: 1.871, acc: 0.591, val_loss: 1.844, val_acc: 0.616
epoch #12 loss: 1.863, acc: 0.600, val_loss: 1.854, val_acc: 0.606
epoch #13 loss: 1.854, acc: 0.607, val_loss: 1.836, val_acc: 0.625
epoch #14 loss: 1.849, acc: 0.614, val_loss: 1.820, val_acc: 0.639
epoch